## Lab2. Set up your property.

[akamai property cli-onboard](https://github.com/akamai/cli-onboard)

In [28]:
# set environment variables
ctr_id=$(akamai pm lg -f json | jq -r .[0].contractIds[0])
grp_id=$(akamai pm lg -f json | jq -r .[0].groupId)
prd_id=$(akamai pm lp -c $ctr_id -f json | jq -r .[0].productId)

prefix=$(python3 src/createName.py)
p_name=$prefix'.akamai-lab.com'
eh_name=$p_name'.edgesuite.net'

c_secret=$(awk '/client_secret/ {print$3}' src/credential.txt | sort -u)
host=$(awk '/host/ {print$3}' src/credential.txt | sort -u)
a_token=$(awk '/access_token/ {print$3}' src/credential.txt | sort -u)
c_token=$(awk '/client_token/ {print$3}' src/credential.txt | sort -u)

echo 'Your property name:' $p_name
echo 'Your CP code name:' $prefix
echo 'Your hostname:' $p_name
echo 'Your edge hostname:' $eh_name

Your property name: 53206-acidic-fleet.akamai-lab.com
Your CP code name: 53206-acidic-fleet
Your hostname: 53206-acidic-fleet.akamai-lab.com
Your edge hostname: 53206-acidic-fleet.akamai-lab.com.edgesuite.net


Create your [CP code](https://techdocs.akamai.com/onboard/docs/set-up-identity-and-access-api#3-get-your-cp-codes)

In [30]:
akamai pm create-cpcode -c $ctr_id -g $grp_id -d $prd_id -n $p_name -f json

In [31]:
cp_id=$(akamai pm list-cpcodes -c $ctr_id -g $grp_id -f json | jq  -r --arg name $p_name  '.[] | select(.cpcodeName == $name) | .cpcodeId' | cut -c 5-)
echo $cp_id

1460175


Create a new property.

In [29]:
akamai property-manager new-property -c $ctr_id -g $grp_id -d $prd_id -p $p_name --insecure 

In [32]:
akamai pm lpr -c $ctr_id -g $grp_id

╒══════════════════╤═══════════════╤════════════╤══════════════════════════════╤═════════════╤══════════════╤════════════════════╤════════════════╤═════════════════╕
│"Account Id"      │"Contract Id"  │"Group Id"  │"Property Name"               │"Property Id"│"Asset Id"    │"Production Version"│"Latest Version"│"Staging Version"│
╞══════════════════╪═══════════════╪════════════╪══════════════════════════════╪═════════════╪══════════════╪════════════════════╪════════════════╪═════════════════╡
│"act_F-AC-4917414"│"ctr_V-41DUHPB"│"grp_232865"│"20230403123732.akamai-lab.com│"prp_907470" │"aid_11149225"│null                │1               │1                │
│                  │               │            │"                             │             │              │                    │                │                 │
├──────────────────┼───────────────┼────────────┼──────────────────────────────┼─────────────┼──────────────┼────────────────────┼────────────────┼─────────────────┤
│"ac

Update main.json and Augmented_insights.json

Edit **main.json** in *workspaces > lab2 > $p_name > config-snippets*.

Update "hostname": "origin-XXXX" to your origin hostname. For example, "origin--main--20230405030614--a.b.akamai-lab.com"

Change "forwardHostHeader": *"REQUEST_HOST_HEADER"* to "forwardHostHeader": *"ORIGIN_HOSTNAME"*

**screencast placeholder**

Edit **Augment_insights.json** in *workspaces > lab2 > test001 > config-snippets* directory.

Update "cpCode",
        "options":{}
        
to
        
          "options": {
            "value": {
              "id": $cp_id
            }
          }

main.json - origin hostname

Augmented_insights.json - cp code
{
    "name": "cpCode",
    "options": {}
}

{
                "name": "cpCode",
                "options": {
                    "value": {
                        "id": 1459812,
                        "name": ""
                    }
                }
            }

In [35]:
# create edge hostname, add property hostname
akamai pm save -p $p_name

╒════════════════════════╤════════╕
│"Action"                │"Result"│
╞════════════════════════╪════════╡
│"stored rule tree"      │"yes"   │
├────────────────────────┼────────┤
│"edge hostnames created"│"no"    │
├────────────────────────┼────────┤
│"stored hostnames"      │"no"    │
├────────────────────────┼────────┤
│"validation warnings"   │"yes"   │
├────────────────────────┼────────┤
│"validation errors"     │"no"    │
├────────────────────────┼────────┤
│"hostname warnings"     │"no"    │
├────────────────────────┼────────┤
│"hostname errors"       │"no"    │
└────────────────────────┴────────┘
There are validation warnings: 
 [
    {
        "type": "https://problems.luna.akamaiapis.net/papi/v0/validation/validation_message.general_warning",
        "errorLocation": {
            "template": "config-snippets/Augment_insights.json",
            "variables": [],
            "location": "children/2/criteria/0",
            "value": {
                "name": "requestType",
     

In [36]:
# Activate property. This can take a couple of minutes.
akamai pm activate -p $p_name -m 'first version' -n staging -w

Following activations are now pending:
╒══════════════════════════════╤═════════╤═══════════════╕
│"Property"                    │"Network"│"Activation Id"│
╞══════════════════════════════╪═════════╪═══════════════╡
│"53206-acidic-fleet.akamai-lab│"STAGING"│12930306       │
│.com"                         │         │               │
└──────────────────────────────┴─────────┴───────────────┘
...activation status is PENDING...
...Waiting for active status...
...Checking activations...
...activation status is PENDING...
...Waiting for active status...
...Checking activations...
...activation status is PENDING...
...Waiting for active status...
...Checking activations...
186.315 seconds since command ran
Activation status report:
╒═══════════════════════════════════╤═════════╤═══════════════╤════════╕
│"Property"                         │"Network"│"Activation Id"│"Status"│
╞═══════════════════════════════════╪═════════╪═══════════════╪════════╡
│"53206-acidic-fleet.akamai-lab.com"│"STAGING"

Visit [your website](http://53206-acidic-fleet.akamai-lab.com), which is akamaized on Akamai staging network.

screencast placeholder

Edit **Augment_insights.json** in *workspaces > lab2 > test001 > config-snippets* directory.

Update "cpCode",
        "options":{}
        
to
        
          "options": {
            "value": {
              "name": "test001",
            }
          }

screencast placeholder